In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 20 13:50:48 2025

@author: nelso
"""

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from scipy.special import comb
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import webbrowser

#Function to calculate the probability to have exactly ka A and exactly kb B
def prob_at_least_ka_and_kb(KA, KB, total_cards, hand_size, ka, kb):
    total_hands = comb(total_cards, hand_size)
    other_cards = total_cards - KA - KB
    
    prob_sum = np.zeros_like(KA, dtype=float)
    for i in range(KA.shape[0]):
        for j in range(KA.shape[1]):
            prob = 0
            for xa in range(ka, min(KA[i, j], hand_size) + 1):
                for xb in range(kb, min(KB[i, j], hand_size - xa) + 1):
                    prob += (comb(KA[i, j], xa) * comb(KB[i, j], xb) * comb(other_cards[i, j], hand_size - xa - xb)) / total_hands
            prob_sum[i, j] = prob
    
    return prob_sum

#We use Dash
app = dash.Dash(__name__)

# Layout of the application
app.layout = html.Div([
    html.H1("Distribution des Probabilités - Au Moins ka et kb"),
    dcc.Graph(id='3d-plot', style={'height': '800px'}),  #Upgrade the size of the plot
    html.Label("Total Cards"),
    dcc.Slider(id='total-cards', min=40, max=60, step=1, value=40, marks={i: str(i) for i in range(40, 61, 5)}),
    html.Label("Hand Size"),
    dcc.Slider(id='hand-size', min=0, max=60, step=1, value=15, marks={i: str(i) for i in range(0, 61, 5)}),
    html.Label("ka"),
    dcc.Slider(id='ka', min=0, max=11, step=1, value=6, marks={i: str(i) for i in range(0, 11)}),
    html.Label("kb"),
    dcc.Slider(id='kb', min=0, max=5, step=1, value=0, marks={i: str(i) for i in range(0, 6)})
])

# Callback to update the plot
@app.callback(
    Output('3d-plot', 'figure'),
    [Input('total-cards', 'value'),
     Input('hand-size', 'value'),
     Input('ka', 'value'),
     Input('kb', 'value')]
)
def update_plot(total_cards, hand_size, ka, kb):
    KA_values = np.arange(0, total_cards + 1)
    KB_values = np.arange(0, total_cards + 1)
    KA, KB = np.meshgrid(KA_values, KB_values)
    Z = prob_at_least_ka_and_kb(KA, KB, total_cards, hand_size, ka, kb)
    
    fig = go.Figure(data=[go.Surface(z=Z, x=KA_values, y=KB_values, colorscale='Viridis')])
    fig.update_layout(
        scene=dict(
            xaxis_title='Number of copies of A (KA)',
            yaxis_title='Number of copies of B (KB)',
            zaxis_title=f'Probability of drawing at least {ka} A and at least {kb} B',
        ),
        title=f'Evolution of P(exactly {ka} A and exactly {kb} B) depending on KA and KB',
        width=1000,  # Upgrade the weigth
        height=800   # Upgrade the heigth
    )
    return fig

# Open in the browser
if __name__ == '__main__':
    webbrowser.open('http://127.0.0.1:8052')
    app.run(debug=True, port=8052)

ObsoleteAttributeException: app.run_server has been replaced by app.run